In [230]:
import os
import pandas as pd
import numpy as np

In [231]:
os.path.realpath('..')

'/Users/akshay/Dropbox/Personal/Data Science/Codes/Projects'

In [232]:
path = '/Users/akshay/Dropbox/Personal/Data Science/Codes/Projects Non Git/Wikisql/WikiSQL/data'

In [233]:
tables_path = os.path.join(path, 'dev.tables.jsonl')
questions_path = os.path.join(path, 'dev.jsonl')

In [234]:
tables = pd.read_json(tables_path, lines=True)
questions = pd.read_json(questions_path, lines=True)

In [275]:
##### FUNCTIONS ######

def get_small_table(table_id):
    details = tables[tables['id']==table_id]
    rows = details['rows'].iloc[0]
    labels = details['header'].iloc[0]
    df = pd.DataFrame(rows, columns=labels)
    return df

def get_cols(table_id):
    return list(get_small_table(table_id).columns)

def add_cols(questions, tables):
    questions['cols'] = questions.apply(lambda row: get_cols(row['table_id']), axis=1)
    questions['qtokens'] = [i.split() for i in questions['question']]
    questions['input_tokens'] = questions['cols'] + questions['qtokens']
    questions.drop(['qtokens'], axis=1, inplace=True)
    return questions

In [237]:
#Get a table id at random
table_id = np.random.choice(meta['table_id'])
table_id

'2-13553506-4'

In [238]:
#Get corresponding dataset
small_df = get_table(table_id)
small_df.head()

,Date,Opponent,Score,Loss,Save
0,June 1,Astros,6 - 5 10,DiPino (1-1),Smith (9)
1,June 2,Astros,13 - 2,Ryan (2-5),"|| 13,891 ||29-21||"
2,June 3,Astros,22 - 7,Knepper (2-6),"|| 19,725 ||30-21||"
3,June 4,Cardinals,3 - 1,Maddux (4-5),Worrell (12)
4,June 5,Cardinals,5 - 1,Sanderson (3-2),"|| 36,818 ||30-23||"


In [282]:
###########################

In [243]:
# training_data = small_df.transpose().reset_index()
# training_data

In [281]:
#############################

In [276]:
questions_new = add_cols(questions, tables)

In [277]:
questions_new.head()

,phase,question,sql,table_id,cols,input_tokens
0,1,What position does the player who played for b...,"{'sel': 3, 'conds': [[5, 0, 'Butler CC (KS)']]...",1-10015132-11,"[Player, No., Nationality, Position, Years in ...","[Player, No., Nationality, Position, Years in ..."
1,1,How many schools did player number 3 play at?,"{'sel': 5, 'conds': [[1, 0, '3']], 'agg': 3}",1-10015132-11,"[Player, No., Nationality, Position, Years in ...","[Player, No., Nationality, Position, Years in ..."
2,1,What school did player number 21 play for?,"{'sel': 5, 'conds': [[1, 0, '21']], 'agg': 0}",1-10015132-11,"[Player, No., Nationality, Position, Years in ...","[Player, No., Nationality, Position, Years in ..."
3,1,Who is the player that wears number 42?,"{'sel': 0, 'conds': [[1, 0, '42']], 'agg': 0}",1-10015132-11,"[Player, No., Nationality, Position, Years in ...","[Player, No., Nationality, Position, Years in ..."
4,1,What player played guard for toronto in 1996-97?,"{'sel': 0, 'conds': [[3, 0, 'Guard'], [4, 0, '...",1-10015132-11,"[Player, No., Nationality, Position, Years in ...","[Player, No., Nationality, Position, Years in ..."


In [280]:
questions_new.iloc[0]['input_tokens']

['Player',
 'No.',
 'Nationality',
 'Position',
 'Years in Toronto',
 'School/Club Team',
 'What',
 'position',
 'does',
 'the',
 'player',
 'who',
 'played',
 'for',
 'butler',
 'cc',
 '(ks)',
 'play?']

In [306]:
all_cols = [k for j in [i for i in tables['header']] for k in j]
len(all_cols)

17130

In [312]:
unique_cols = np.unique([k for j in [i for i in tables['header']] for k in j], return_counts=True)

In [311]:
from sklearn.feature_extraction.text import CountVectorizer

In [332]:
cnt1 = CountVectorizer(stop_words=None, vocabulary=unique_cols[0])
col_encodings = cnt1.fit_transform([' '.join(i) for i in questions_new['cols']])
col_encodings

<8421x4025 sparse matrix of type '<class 'numpy.int64'>'
	with 4568 stored elements in Compressed Sparse Row format>

In [333]:
cnt2 = CountVectorizer(stop_words='english')
ques_encodings = cnt2.fit_transform(questions_new['question'])
ques_encodings

<8421x8451 sparse matrix of type '<class 'numpy.int64'>'
	with 44772 stored elements in Compressed Sparse Row format>

In [339]:
inp1 = Input((col_encodings.shape[1],))
inp2 = Input((ques_encodings.shape[1],))
x = Concatenate((inp1, inp2))

In [336]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Concatenate

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, 1000))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

NameError: name 'latent_dim' is not defined

In [285]:
n_input = 1000

In [295]:
encoder_inputs = Input(shape=(None, n_input))
encoder = LSTM(32, return_state=True)(encoder_inputs)
#encoder_states = [state_h, state_c]

In [294]:
encoder

<tf.Tensor 'lstm_3/TensorArrayReadV3:0' shape=(?, 32) dtype=float32>